In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)

template = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b}"
)

prompt = template.format(country_a="Mexico", country_b="Thailand")


chat.predict(prompt)

'The distance between Mexico and Thailand is approximately 9,500 miles (15,300 kilometers) when measured in a straight line.'

In [8]:
template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert. And you only reply in {language}."),
        ("ai", "Ciao, mi chiamo {name}!"),
        (
            "human",
            "What is the distance between {contry_a} and {contry_b}. Also, what is your name?",
        ),
    ]
)

prompt = template.format_messages(
    language="English",
    name="Seokwoo",
    contry_a="Mexico",
    contry_b="Thailand",
)

chat.predict_messages(prompt)

AIMessage(content='Hello! The distance between Mexico and Thailand is approximately 16,000 kilometers (9,942 miles) when measured in a straight line. My name is Seokwoo. How can I assist you today?')

In [11]:
from langchain.schema import BaseOutputParser


class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))

['Hello', 'how', 'are', 'you']

In [15]:
template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items}. Do NOT reply with anything else.",
        ),
        (
            "human",
            "{question}",
        ),
    ]
)

prompt = template.format_messages(max_items=10, question="What are the planets?")

result = chat.predict_messages(prompt)

p = CommaOutputParser()

p.parse(result.content)

['Mercury', 'Venus', 'Earth', 'Mars', 'Jupiter', 'Saturn', 'Uranus', 'Neptune']

In [17]:
chain = template | chat | CommaOutputParser()

chain.invoke({"max_items": 10, "question": "What are the planets?"})

['Mercury',
 'Venus',
 'Earth',
 'Mars',
 'Jupiter',
 'Saturn',
 'Uranus',
 'Neptune',
 'Pluto']

In [ ]:
chef_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a chef. You only reply in {language}."),
        ("human", "I want to make {food}."),
    ]
)

veg_chef_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a vegetarian chef. You only reply in {language}."),
        ("human", "I want to make {food}."),
    ]
)

chef_chain = chef_prompt | chat

veg_chain = veg_chef_prompt | chat

final_chain = chef_chain | veg_chain